In [ ]:
import langchain
import requests
import functools
from threading import Thread
from langchain import PromptTemplate
from langchain.chat_models import ChatOpenAI
from langchain.utilities import GoogleSerperAPIWrapper
from langchain.document_loaders import UnstructuredURLLoader
from langchain.embeddings import OpenAIEmbeddings
from langchain.vectorstores import FAISS
from langchain.output_parsers import RegexParser
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.chains.qa_with_sources import load_qa_with_sources_chain
from bs4 import BeautifulSoup
from dotenv import load_dotenv

load_dotenv()

In [ ]:
# How many similar articles to use as reference
SIMILAR_COUNT = 3
# Original article url
ARTICLE_URL = "https://www.foxnews.com/opinion/biden-views-debt-ceiling-staring-contest-republicans-duty-president"

In [ ]:
response = requests.get(ARTICLE_URL)
soup = BeautifulSoup(response.text, 'html.parser')
title = soup.find('title').get_text().split('|')[0]

In [ ]:
# serp_tool = GoogleSerperAPIWrapper(tbs="qdr:m")
# similar_articles_serp = serp_tool.results(f"{title} news articles")

In [ ]:
# similar_articles_final = []
# for article_meta in similar_articles_serp["organic"]:
#     link = article_meta["link"]
#     if not link.startswith('https://www.youtube.com') and not link.startswith('https://youtube.com'):
#         similar_articles_final.append(link)
#         if len(similar_articles_final) == SIMILAR_COUNT:
#             break
# print(similar_articles_final)

In [ ]:
total_articles = [ARTICLE_URL]
# total_articles.extend(similar_articles_final)

In [ ]:
# https://stackoverflow.com/questions/21827874/timeout-a-function-windows
def timeout(timeout):
    def deco(func):
        @functools.wraps(func)
        def wrapper(*args, **kwargs):
            res = [Exception('function [%s] timeout [%s seconds] exceeded!' % (func.__name__, timeout))]
            def newFunc():
                try:
                    res[0] = func(*args, **kwargs)
                except Exception as e:
                    res[0] = e
            t = Thread(target=newFunc)
            t.daemon = True
            try:
                t.start()
                t.join(timeout)
            except Exception as je:
                print ('error starting thread')
                raise je
            ret = res[0]
            if isinstance(ret, BaseException):
                raise ret
            return ret
        return wrapper
    return deco

In [ ]:
loader = UnstructuredURLLoader(urls=total_articles)
try:
    articles_data = timeout(timeout=15)(loader.load)()
except:
    print("Timeout")

In [ ]:
medium_text_splitter = RecursiveCharacterTextSplitter(
    chunk_size = 500,
    chunk_overlap  = 75,
    length_function = len
)

In [ ]:
# article_documents = []
# for article_content in articles_data[1:]:
#     article_documents.extend(medium_text_splitter.create_documents([article_content.page_content]))

In [ ]:
primary_texts = medium_text_splitter.split_text([articles_data[0].page_content])

In [ ]:
# embeddings = OpenAIEmbeddings()
# article_db = FAISS.from_documents(article_documents, embeddings)

In [ ]:
# article_retriever = article_db.as_retriever()
# article_retriever.get_relevant_documents("")

In [ ]:
primary_embeddings = OpenAIEmbeddings()
docsearch = FAISS.from_texts(primary_texts, primary_embeddings, metadatas=[{"source": str(i)} for i in range(len(primary_texts))])
docs = docsearch.similarity_search(title)

In [ ]:
prompt_template = """Use the following excerpt from a news article titled "{question}" to determine whether any signals of political bias, political narratives, misinformation, or opinion-based journalism practices exist within the news article.

If you detect any signals of political bias, signals of a political narrative being pushed, signals of misinformation, or signals opinion-based journalism practices, concisely explain what the signals are. If you can't find any signals, reply with "None found.", don't try to make up signals.

In addition to giving an explanation of the signals, also return a score of how confident you are that political bias, political narratives, misinformation, or opinion-based journalism exists in the excerpt. This should be in the following format:

Signals: [explination of signals here]
Score: [confidence score between 0 and 100](Only return a number for the score, no other text is allowed.)

Begin!

excerpt:
---------
{context}
---------
Signals:"""

In [ ]:
output_parser = RegexParser(
    regex=r"(.*?)\nScore: (.*)",
    output_keys=["answer", "score"],
)

PROMPT = PromptTemplate(
    template=prompt_template,
    input_variables=["context", "question"],
    output_parser=output_parser,
)

In [ ]:
chain = load_qa_with_sources_chain(ChatOpenAI(model_name='gpt-3.5-turbo', temperature=0.0), chain_type="map_rerank", metadata_keys=['source'], return_intermediate_steps=False, prompt=PROMPT)
result = chain({"input_documents": docs, "question": title}, return_only_outputs=False)

In [ ]:
if result["output_text"] == 'None found.':
    print("No Signals of Political Bias Found.")

In [ ]:
for doc in result['input_documents']:
    if doc.metadata['source'] == result['source']:
        # print(doc.page_content)
        print(result["output_text"])

In [ ]:
# Future Addition: Use article_retriever.get_relevant_documents() to find similar excerpt from other articles and recommend an alternative based on the least biased by comparison